# How to load the wav file

In [1]:
import tensorflow as tf
import numpy as np
import glob
import librosa

data1 = []
labels1 = []
meta1=[]

aorticstenosis = glob.glob('/kaggle/input/15sdataforheartsound/aorticstenosis/*.wav')
mitral_regurgitation = glob.glob('/kaggle/input/15sdataforheartsound/mitral regurgitation/*.wav')
mitral_stenosis = glob.glob('/kaggle/input/15sdataforheartsound/mitral stenosis/*.wav')
mitral_valve_prolapse = glob.glob('/kaggle/input/15sdataforheartsound/mitral valve prolapse/*.wav')
normal = glob.glob('/kaggle/input/15sdataforheartsound/normal/*.wav')

SAMPLE_RATE=1000


for file_path in aorticstenosis:   
    #print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal)
    data1.append(signal)
    labels1.append(0)

for file_path in mitral_regurgitation:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal)
    data1.append(signal)
    labels1.append(1)
    
for file_path in mitral_stenosis:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal)
    data1.append(signal)
    labels1.append(2)
    
for file_path in mitral_valve_prolapse:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal)
    data1.append(signal)
    labels1.append(3)
    
for file_path in normal:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal)
    data1.append(signal)
    labels1.append(4)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
X= np.array(data1)
y= np.array(labels1)

In [3]:
X.shape

(1000, 1125)

In [4]:
y.shape

(1000,)

In [16]:
from sklearn.model_selection import train_test_split
num_classes = 5

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, tf.keras.utils.to_categorical(y, num_classes), test_size=0.2, random_state=32)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=32)


640


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Generate synthetic training data
num_samples = 1000
input_shape = (1125,1)

# Define the LSTM-based model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, activation= 'relu',input_shape=input_shape, return_sequences= True),
    tf.keras.layers.LSTM(100),
    
    tf.keras.layers.Dense(750, activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Monitor validation loss for learning rate reduction
    factor=0.05,            # Reduce learning rate by a factor of 0.1
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6            # Minimum learning rate
)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=6)

# Train the model

model.fit(X_train, y_train,validation_data=(X_val,y_val),epochs=100,batch_size=32,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model.h5')
loss, accuracy = best_model.evaluate(X_test, y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")


Epoch 1/100
20/20 [==============================] - 25s 1s/step - loss: 1.5799 - accuracy: 0.2391 - val_loss: 1.5828 - val_accuracy: 0.3250 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 21s 1s/step - loss: 1.5081 - accuracy: 0.3281 - val_loss: 1.5287 - val_accuracy: 0.3313 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 20s 985ms/step - loss: 1.4554 - accuracy: 0.3359 - val_loss: 1.5079 - val_accuracy: 0.3375 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 20s 1s/step - loss: 1.4254 - accuracy: 0.3734 - val_loss: 1.5065 - val_accuracy: 0.3125 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 19s 964ms/step - loss: 1.4216 - accuracy: 0.3391 - val_loss: 1.4601 - val_accuracy: 0.3500 - lr: 0.0010
Epoch 6/100
20/20 [==============================] - 21s 1s/step - loss: 1.3778 - accuracy: 0.3828 - val_loss: 1.4814 - val_accuracy: 0.3562 - lr: 0.0010
Epoch 7/100
20/20 [==============================] - 20s 1s/step - los

# Implement all the steps 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Generate synthetic training data
num_samples = 1000
input_shape = (1125, 1)

# Define the LSTM-based model
model = Sequential([
    LSTM(256, input_shape=input_shape, return_sequences=True),
    LSTM(256),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define callbacks
lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[lr_callback, checkpoint_callback, early_stopping_callback]
)

# Load the best model
best_model = tf.keras.models.load_model('best_model.h5')

# Evaluate the model on the test set
loss, accuracy = best_model.evaluate(X_test, y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")